## Load Modules

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append('/home/zachpen87/clearmap/ClearMap2/')

import h5py
import holoviews as hv
import dask.array as da
import numpy as np
import scipy.ndimage as ndimage
import scipy.stats as stats
import skimage
from skimage import io
#import ClearMap.ImageProcessing.H5 as H5img
import ClearMap.Visualization.JupNtbk_vis as jvis
hv.notebook_extension('bokeh')

## Define directory and group information

In [ ]:
channel = 'fos_790'
directory = '/home/zachpen87/clearmap/data/SEFL17b/' 
res_directory = '/home/zachpen87/clearmap/data/SEFL17b/results' 

group_dict = {
    'NoStim_creneg' : ['ms_27','ms_42','ms_53','ms_71'],
    'NoStim_cre' : ['ms_103','ms_105','ms_106'],
    'NoStim' : ['ms_103','ms_105','ms_106','ms_107','ms_27','ms_42','ms_53','ms_71'],
    'NoTrauma' : ['ms_28','ms_37','ms_41','ms_45','ms_46','ms_49','ms_65','ms_72','ms_85','ms_88'],
    'Trauma' : ['ms_26','ms_35','ms_40','ms_44','ms_47','ms_54','ms_66','ms_74','ms_86'], # ms89 removed
}

if not os.path.isdir(res_directory):
    os.mkdir(res_directory)

## Load ABA reference image and size

In [ ]:
ABA_directory = '/home/zachpen87/clearmap/AtlasDocs/Horizontal'
ABA_ref = 'ABA_25um_reference__1_2_3__slice_None_None_None__slice_None_None_None__slice_None_None_None__.tif'
ref = skimage.io.imread(os.path.join(ABA_directory, ABA_ref))
htmp_shape = ref.shape

## For each group, create single hdf5 with 4d heatmat dataset (animal, Z, Y, X)

In [ ]:
for group in group_dict:
    
    fpath = os.path.join(
        res_directory,
        '.'.join(['_'.join([group,channel]),
                             'hdf5'
                         ]))
    
    with h5py.File(fpath, 'w') as f: 
        
        n = [len(group_dict[group])]
        f.create_dataset('n', data=n, dtype='uint8')
        
        f.create_dataset(
            'allmaps',
            tuple(np.concatenate([n, list(htmp_shape)])),
            dtype = 'float64')
        
        for idx, ms in enumerate(group_dict[group]):
            print('writing {ms} to group: {group}'.format(ms=ms,group=group))
            subpath = os.path.join(directory,ms,'data.hdf5')
            with h5py.File(subpath, 'r') as subf:
                f['allmaps'][idx,:,:,:] = subf['_'.join([channel, 'heatmap'])][:]
                    

## Define mean and standard deviation heatmaps

In [ ]:
for group in group_dict:
    
    fpath = os.path.join(
        res_directory,
        '.'.join(['_'.join([group,channel]),
                             'hdf5'
                         ]))
    with h5py.File(fpath, 'a') as f: 
        
        print('creating dask array for all data in group {group}'.format(group=group))
        n=f['n'][0]
        allmaps = da.from_array(
            f['allmaps'],
            chunks=(n,500,500,500))
        
        print('calculating mean heatmap for group: {group}'.format(group=group))
        allmaps.mean(axis=0).to_hdf5(fpath,('/'+'mean'))
        
        print('calculating median heatmap for group: {group}'.format(group=group))
        da.median(allmaps, axis=0).to_hdf5(fpath,('/'+'median'))
        
#         print('calculating std dev heatmap for group: {group}'.format(group=group))
#         allmaps.std(axis=0).to_hdf5(fpath,('/'+'std'))
        
#         print('calculating z heatmap for group: {group}'.format(group=group))
#         z = f['mean'][:]
#         std = f['std'][:]
#         z[std==0] = 0
#         z[std>0] = z[std>0]/std[std>0]
#         f.create_dataset('z',data=z)
        
        

## Visualize group heatmaps

In [ ]:
%output size = 130

groups = ['NoTrauma','Trauma']
dset = 'median'
interval = 5
plane = 'c'
include_ref = True

hmap_dict = {}
if include_ref:
    hmap_dict['ref'] = jvis.gen_hmap(ref,plane=plane,title='Allen Brain Atlas',inter=interval,tools=['hover']) 

for group in groups:
    hdf5_file = os.path.join(res_directory, '{group}_{channel}.hdf5'.format(group=group,channel=channel))
    with h5py.File(hdf5_file,'r') as f:
        hmap_dict[group] = jvis.gen_hmap((f[dset][:]*100)**2,
                               plane=plane,
                               title= '{group}: {var}'.format(group=group,var=dset),
                               inter=interval,lims=(0,50),
                               cmap='inferno',
                               alpha=1,
                               tools=['hover'])
    
hv.NdLayout(hmap_dict).cols(1)